In [1]:
from datasets import load_dataset
import datasets
from transformers import (
    AutoTokenizer,
    AutoConfig,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
)
import torch
import numpy as np

In [3]:
# load dataset
ag_dataset = load_dataset('ag_news')

# Create a train/dev/test splits
ag_dev_dataset = load_dataset('ag_news', split='train[10%:11%]')
ag_train_dataset = load_dataset('ag_news', split='train[:10%]')
ag_test_dataset = load_dataset('ag_news', split='test[11%:12%]')

# merge the splits in a single `datasets.DatasetDict` object
ag_split = {split: data for split, data in zip(['train', 'test', 'dev'], [ag_train_dataset, ag_test_dataset, ag_dev_dataset])}
ag_dataset_split =  datasets.DatasetDict(ag_split)

# Count the number of labels.
# Important: use all the splits to compute the labels. 
num_labels = len(set(ag_dataset_split['train'].unique('label') + 
                     ag_dataset_split['test'].unique('label') +
                     ag_dataset_split['dev'].unique('label')))

Using custom data configuration default
Reusing dataset ag_news (/home/deniz/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548)
Using custom data configuration default
Reusing dataset ag_news (/home/deniz/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548)
Using custom data configuration default
Reusing dataset ag_news (/home/deniz/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548)
Using custom data configuration default
Reusing dataset ag_news (/home/deniz/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548)


In [6]:
model_pretrained = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_pretrained)

"""🤔 **Understanding BERT tokenizer**"""

# Define a tokenization function for the dataset used a standard for text classification
def tokenize(dataset):
    sentences = dataset['text']
    return tokenizer(sentences, 
                     padding='max_length',
                     truncation=True)
    
# apply it 
ag_dataset_tokenized = ag_dataset_split.map(tokenize,
                                            batched=True,
                                            remove_columns=['text'],
                                            desc='Tokenize data')

Loading cached processed dataset at /home/deniz/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548/cache-202c20a3ee59b4d0.arrow
Tokenize data: 100%|██████████| 1/1 [00:00<00:00, 13.46ba/s]
Loading cached processed dataset at /home/deniz/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548/cache-0a9fc35bce44a8ca.arrow


In [7]:
# load model
from transformers import BertForSequenceClassification

checkpoint = 'bert-ag-news-classification'
model = BertForSequenceClassification.from_pretrained(checkpoint, num_labels=num_labels)

In [32]:
test_data = ag_dataset_tokenized['test']
display(test_data)
print(type(test_data))

Dataset({
    features: ['attention_mask', 'input_ids', 'label', 'token_type_ids'],
    num_rows: 76
})

<class 'datasets.arrow_dataset.Dataset'>


In [31]:

X = test_data.pop['label']
y = test_data['label']



AttributeError: 'Dataset' object has no attribute 'pop'

In [20]:
test_trainer = Trainer(model) 

In [33]:
raw_pred = test_trainer.predict(test_data) 

***** Running Prediction *****
  Num examples = 76
  Batch size = 8
 90%|█████████ | 9/10 [00:01<00:00,  7.40it/s]

In [34]:
y_true = test_data['label']

In [ ]:
# Step 5: Making prediction
######## Tokenize test data

#X_test_tokenized = tokenizer(X_test, padding=True, truncation=True, max_length=512)
#  
########## Create torch dataset

#test_dataset = Dataset(X_test_tokenized) 

############ Load trained model

#model_path = "output/checkpoint-50000"
#model = BertForSequenceClassification.from_pretrained(model_path, num_labels=2) 

############## Define test trainer

#test_trainer = Trainer(model) 

############ Make prediction

#raw_pred, _, _ = test_trainer.predict(test_dataset) 

############ Preprocess raw predictions

#y_pred = np.argmax(raw_pred, axis=1)